In [0]:
caminho_entrada = "/FileStore/dados/PORTES_2025.csv"
caminho_saida = "/FileStore/julianams/PORTES_2025_delta"

In [0]:
# BLOCO 1 - LEITURA

'''df = spark.read.format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option('encoding', 'ISO-8859-1') \
    .load(caminho_entrada) 

display(df)'''

df = spark.read.format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option('sep', ';') \
    .option('encoding', 'ISO-8859-1') \
    .load(caminho_entrada)


In [0]:
# BLOCO 3 - LIMPEZA

from pyspark.sql.functions import col

df_limpo = df \
    .dropna(subset=["TOTAL"]) \
    .fillna("NÃO INFORMADO", subset=["UF", "MUNICIPIO"]) \
    .filter(col("TOTAL") > 0)

In [0]:
# BLOCO 4 - GRAVAÇÃO NO DELTA

df_limpo.coalesce(1) \
  .write.format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .save(caminho_saida)

# VALIDAÇÃO
df_delta = spark.read.format("delta").load(caminho_saida)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS hive_metastore.julianams;

In [0]:
%sql

DROP TABLE hive_metastore.julianams.portes

In [0]:
%sql

CREATE TABLE hive_metastore.julianams.portes
USING DELTA
LOCATION '/FileStore/julianams/PORTES_2025_delta';

In [0]:
%sql
select * from hive_metastore.julianams.portes;